3. DATA PREPARATION


Importing relevant libraries

In [264]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer


Loading the Dataset

In [265]:
df=pd.read_csv("judge-1377884607_tweet_product_company.csv",encoding='ISO-8859-1')#since the default encoding used by pandas(utf-8) could not decode the byte in the csv filr
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [266]:
#Checking the Dataset status
class DataFrameChecker:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def check_info(self):
        print("DataFrame Info:")
        self.dataframe.info()

    def check_shape(self):
        print(f"DataFrame Shape: {self.dataframe.shape}")

    def check_describe(self):
        print("DataFrame Description:")
        print(self.dataframe.describe())
    def check_nulls(self):
        print("Null Values in DataFrame:")
        print(self.dataframe.isnull().sum())

    def check_duplicates(self):
        duplicate_rows = self.dataframe[self.dataframe.duplicated()]
        print(f"Number of Duplicate Rows: {duplicate_rows.shape[0]}")
        if duplicate_rows.shape[0] > 0:
            print("Duplicate Rows:")
            print(duplicate_rows)


    # Create an instance of DataFrameChecker
checker = DataFrameChecker(df)

    # Use the methods
print(checker.check_info())
print(checker.check_shape())
print(checker.check_describe())
print(checker.check_nulls())
print(checker.check_duplicates())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB
None
DataFrame Shape: (9093, 3)
None
DataFrame Description:
                                               tweet_text  \
count                                                9092   
unique                                               9065   
top     RT @mention Marissa Mayer: Google Will Connect...   
freq                                                    5   

       emotion_in_tweet_is_directed_at  \
count                       

3.1 Data Cleaning "emotion_in_tweet_is_directed_at" Column

In [267]:
df["emotion_in_tweet_is_directed_at"].value_counts()

emotion_in_tweet_is_directed_at
iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: count, dtype: int64

Categorizing the brands to either Apple or Google products

In [268]:
product_category={
    "iPhone":"Apple", 
    "iPad":"Apple",
    "iPad or iPhone App":"Apple", 
    "Other Apple product or service": "Apple",
    "Android":"Google", 
    "Android App":"Google", 
    "Other Google product or service":"Google"
}
df["brand"]=df["emotion_in_tweet_is_directed_at"].replace(product_category)

               
print(df["brand"].value_counts())
print(df.info())

brand
Apple     2409
Google     882
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   brand                                               3291 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB
None


Dealing with missing values in the brand column

In [269]:
filtered_df=df[df["brand"].isna()]
filtered_df.head(100)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,brand
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product,NaN
6,NaN,NaN,No emotion toward brand or product,NaN
16,Holler Gram for iPad on the iTunes App Store -...,NaN,No emotion toward brand or product,NaN
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,No emotion toward brand or product,NaN
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product,NaN
...,...,...,...,...
186,Currently 150 people in line at the &quot;Pop ...,NaN,No emotion toward brand or product,NaN
187,Only iPad 2 available at #sxsw is the 64GB wif...,NaN,No emotion toward brand or product,NaN
188,÷¼ We love 2 entertain youÛ_Please donÛªt b...,NaN,No emotion toward brand or product,NaN
189,Less than 2 hours until we announce the detail...,NaN,No emotion toward brand or product,NaN


Dropping the 'emotion_in_tweet_is_directed_at' column since the 'brand' column is a replacement of it.

In [270]:
df.drop(columns=['emotion_in_tweet_is_directed_at'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 2   brand                                               3291 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


From the filtered_df there are words in the tweet_text column that can help identify the brand.
For example, if the tweet_text contains the word "iPhone," it's likely an Apple brand. Similarly, if the tweet_text contains the word "Android," it's likely a Google brand.

In [271]:
class KeywordBrandAssigner:
    def __init__(self, keyword_to_brand):
        self.keyword_to_brand = keyword_to_brand  # Mapping from keywords to brands

    def assign_brands(self, df):
        for index, row in df.iterrows():
            text = row['tweet_text']
            if pd.isna(row['brand']) and isinstance(text, str):
                text_lower = text.lower()
                for keyword, brand in self.keyword_to_brand.items():
                    if keyword in text_lower:
                        df.at[index, 'brand'] = brand
                        break
        # Fill any remaining NaN values with 'Unknown'
        df['brand'] = df['brand'].fillna('Unknown')

# Example usage
keyword_to_brand = {
    'google': 'Google',
    'apple': 'Apple',
    'ipad': 'Apple',
    'itunes': 'Apple',
    'android': 'Google',
    'iphone': 'Apple'
}

assigner = KeywordBrandAssigner(keyword_to_brand)
assigner.assign_brands(df)


The KeywordBrandAssigner class maps specific keywords to their corresponding brands ("Apple" or "Google") based on the keyword_to_brand dictionary and assigns these brands to the 'brand' column of the DataFrame when a keyword is found in the tweet text. It ensures that any missing or unmatched values in the 'brand' column are filled with "Unknown.

In [272]:
#reloading the dataset
df["brand"].value_counts()

brand
Apple      5440
Google     2946
Unknown     707
Name: count, dtype: int64

In [273]:
#Dropping the unknown 
df = df[df['brand'] != 'Unknown']
df['brand'].value_counts()

brand
Apple     5440
Google    2946
Name: count, dtype: int64

Dropped the rows with unknown values since i'm focussed with tweets having information about either Google or Apple

In [274]:
#Reloading the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8386 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          8386 non-null   object
 1   is_there_an_emotion_directed_at_a_brand_or_product  8386 non-null   object
 2   brand                                               8386 non-null   object
dtypes: object(3)
memory usage: 262.1+ KB


Checking and Dropping of Duplicates

In [275]:
print(df.duplicated().value_counts())
print(df.drop_duplicates(inplace=True))
print(df.shape)

False    8366
True       20
Name: count, dtype: int64
None
(8366, 3)


C:\Users\pc\AppData\Local\Temp\ipykernel_7368\1557657433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print(df.drop_duplicates(inplace=True))


In [276]:
df.duplicated().value_counts()

False    8366
Name: count, dtype: int64

3.2 Data Cleaning "is there an emotion directed at a brand or product" column.

In [277]:
df["is_there_an_emotion_directed_at_a_brand_or_product"].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    4688
Positive emotion                      2960
Negative emotion                       568
I can't tell                           150
Name: count, dtype: int64

There are 4 classifiers(no emotion,positive,negative and I can't tell), I need a binary classifier model.

To build a binary classifier, I need to consolidate the existing categorical emotions into two distinct classes. Specifically, I will categorize "Positive emotion" as the positive class, while grouping "No emotion toward brand or product," "Negative emotion," and "I can't tell" into the non-positive class. This transformation will simplify the classification task and enable the development of a binary emotion classifier.

In [278]:
Emotions={
    "No emotion toward brand or product":"Non-Positive",
    "Positive emotion":"Positive",
    "Negative emotion":"Non-Positive",
    "I can't tell":"Non-Positive"
}
df.loc[:,"is_there_an_emotion_directed_at_a_brand_or_product"] = df["is_there_an_emotion_directed_at_a_brand_or_product"].map(Emotions)
               
print(df["is_there_an_emotion_directed_at_a_brand_or_product"].value_counts(normalize=True))




is_there_an_emotion_directed_at_a_brand_or_product
Non-Positive    0.646187
Positive        0.353813
Name: proportion, dtype: float64


Creating a copy of the cleaned data

In [280]:
cleaned_data =df.copy()
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8366 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          8366 non-null   object
 1   is_there_an_emotion_directed_at_a_brand_or_product  8366 non-null   object
 2   brand                                               8366 non-null   object
dtypes: object(3)
memory usage: 261.4+ KB


## EXPLAROTORY DATA ANALYSIS

### Tokenizing

The initial step in preparing the dataset for summarization involves breaking each tweet (currently stored as a text string in the overall table) into individual tokens, separated by commas, and then repopulating each row in the dataframe as a list of tokens. The tokenizer uses a regular expression pattern to split words by non-letter characters while preserving apostrophes within words where relevant. 

Areas of focus for the preprocessing phase:
    i. Tokenizing the tweet texts using 'Tweet Tokenizer' that handles hashtags ('#') and mentions ('@').
    ii. Removing basic stop words.

In [ ]:
# Initializing Tokenizer
tknzr